In [2]:
#  This code just reads the main pendency file 

import pandas as pd
from datetime import datetime
import pytz

time_zone = pytz.timezone('Asia/Kolkata')
current_time= datetime.now(time_zone)

df = pd.read_csv("Pendency_automation_report_ageing_greater_than_12.csv")
df


,bag_facility_source_name,bag_facility_current_name,bag_facility_destination_name,bag_type_ph,tracking_id_ekart,bag_id,bag_status,fact_updated_at,_source,bag_xdock,fact_updated_at_1
0,BareillyHub_BRY_PL,MotherHub_BNL,MotherHub_YKB,B5,FMPC3232301347,PH-B5-DELYKB-29433541,REACHED,2023-11-18T22:26:49.000+05:30,BAG,True,2023-11-18T22:26:49.000+05:30
1,SelaquiHub_DED_PL,MotherHub_BNL,MotherHub_YKB,B5,FMPC3246157966,PH-B5-DELYKB-30545149,REACHED,2023-11-19T00:49:51.000+05:30,BAG,True,2023-11-19T00:49:51.000+05:30
2,CABTJalandharOPH_JLR_PL,MotherHub_YKB,MotherHub_YKB,B5,FMPC3220826670,PH-B5-DELYKB-29778596,OPENED,2023-11-17T21:40:23.000+05:30,BAG,False,2023-11-17T21:40:23.000+05:30
3,MundkaHub_DEL_PL,MotherHub_YKB,MotherHub_YKB,ZO,FMPC3247098655,PH-ZO-DELYKB-30442931,OPENED,2023-11-18T03:43:04.000+05:30,BAG,False,2023-11-18T03:43:04.000+05:30
4,RajnagarMP_DEL_PL,MotherHub_YKB,MotherHub_YKB,ZO,FMPC3245783261,PH-ZO-DELYKB-30537337,OPENED,2023-11-17T23:31:38.000+05:30,BAG,False,2023-11-17T23:31:38.000+05:30
...,...,...,...,...,...,...,...,...,...,...,...
4173,RohtakHub_ROH_PL,MotherHub_BNL,MotherHub_YKB,ZO,FMPC3248375219,PH-ZO-DELYKB-30580149,REACHED,2023-11-19T00:33:23.000+05:30,BAG,True,2023-11-19T00:33:23.000+05:30
4174,RohtakHub_ROH_PL,MotherHub_BNL,MotherHub_YKB,ZO,FMPC3249498528,PH-ZO-DELYKB-30580149,REACHED,2023-11-19T00:33:23.000+05:30,BAG,True,2023-11-19T00:33:23.000+05:30
4175,RohtakHub_ROH_PL,MotherHub_BNL,MotherHub_YKB,ZO,FMPC3246671499,PH-ZO-DELYKB-30580149,REACHED,2023-11-19T00:33:23.000+05:30,BAG,True,2023-11-19T00:33:23.000+05:30
4176,RohtakHub_ROH_PL,MotherHub_BNL,MotherHub_YKB,ZO,FMPC3247689192,PH-ZO-DELYKB-30580149,REACHED,2023-11-19T00:33:23.000+05:30,BAG,True,2023-11-19T00:33:23.000+05:30


In [3]:

df_zone_wise = df.groupby('bag_type_ph').size()
df_zone_wise

bag_type_ph
B5    1782
ZO    2396
dtype: int64

In [4]:

# Segregated ZO PH & SPH in this Cell

df_zo = df[df['bag_type_ph'] == "ZO"]
df_zo
df_zo_ph = df_zo[df_zo["bag_facility_source_name"].str.contains("DEL_PL|Bilaspur|Bhiwani")]
df_zo_ph_wise = df_zo_ph.groupby('bag_facility_source_name').size()
print(sum(df_zo_ph_wise))


df_zo_sph = df_zo[~df_zo.index.isin(df_zo_ph.index)]
df_zo_sph_wise = df_zo_sph.groupby('bag_facility_source_name').size()
print(sum(df_zo_sph_wise))



652
1744


In [5]:
# Segregated B5 PH & SPH in this cell same as above

df_b5 = df[df['bag_type_ph'] == "B5"]
df_b5

df_b5_ph = df_b5[df_b5['bag_facility_source_name'].str.contains("DEL_PL|Bilaspur|Bhiwani")]
df_b5_ph_wise  = df_b5_ph.groupby('bag_facility_source_name').size()
print(sum(df_b5_ph_wise))


df_b5_sph  = df_b5[~df_b5.index.isin(df_b5_ph.index)]
df_b5_sph_wise = df_b5_sph.groupby('bag_facility_source_name').size()
print(sum(df_b5_sph_wise))

344
1438


In [6]:
def calculate_and_categorize_time(dataframe, column_name, current_time  , bucket_name):
    df_time_wise = pd.to_datetime(dataframe[column_name])
    df_hour_wise = (current_time - df_time_wise) / pd.Timedelta(hours=1)
    df_hour_wise = df_hour_wise.round().astype(int)
    
    def categorize_time(hour_diff):
        if hour_diff > 48:
            return '> 48 hours'
        elif hour_diff > 24:
            return '24-48 hours'
        elif hour_diff > 11:
            return '12-24 hours'
        else:
            return '< 12 hours'
    
    dataframe["hour_diff"] = df_hour_wise
    dataframe[bucket_name] = df_hour_wise.apply(categorize_time)
    data = dataframe[bucket_name].value_counts()
    return data


final_zo_ph = calculate_and_categorize_time(df_zo_ph , 'fact_updated_at' , current_time , 'ZO PH')
print(f"{final_zo_ph} & total {sum(final_zo_ph)}" )

final_zo_sph = calculate_and_categorize_time(df_zo_sph , 'fact_updated_at' , current_time , "ZO SPH")
print(f"{final_zo_sph}  && Total:  {sum(final_zo_sph)}")


print(f"ZO Total PH + SPH =   {sum(final_zo_sph) + sum(final_zo_ph)}")


final_b5_ph = calculate_and_categorize_time(df_b5_ph , 'fact_updated_at' , current_time , "B5 PH")
print(f" {final_b5_ph} && Total:  {sum(final_b5_ph)}")

final_b5_sph = calculate_and_categorize_time(df_b5_sph , 'fact_updated_at' , current_time , "B5 SPH")
print(f"{final_b5_sph} && Total {sum(final_b5_sph)}")

print(f"B5 Final PH + SPH =  {sum(final_b5_ph) + sum(final_b5_sph)}")

ZO PH
12-24 hours    249
24-48 hours    247
> 48 hours     156
Name: count, dtype: int64 & total 652
ZO SPH
12-24 hours    1548
24-48 hours     126
> 48 hours       70
Name: count, dtype: int64  && Total:  1744
ZO Total PH + SPH =   2396
 B5 PH
24-48 hours    136
> 48 hours     106
12-24 hours    102
Name: count, dtype: int64 && Total:  344
B5 SPH
12-24 hours    1355
24-48 hours      49
> 48 hours       34
Name: count, dtype: int64 && Total 1438
B5 Final PH + SPH =  1782


/var/folders/_4/bc29t88d5ml7g9p13zkltr6w0000gn/T/ipykernel_73458/1836197375.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["hour_diff"] = df_hour_wise
/var/folders/_4/bc29t88d5ml7g9p13zkltr6w0000gn/T/ipykernel_73458/1836197375.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[bucket_name] = df_hour_wise.apply(categorize_time)
/var/folders/_4/bc29t88d5ml7g9p13zkltr6w0000gn/T/ipykernel_73458/1836197375.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic